In [16]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score


In [17]:
df = pd.read_parquet(r"data/df_kpi_perf_2108_1510.parquet", engine="pyarrow")

In [18]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2095147 entries, 0 to 2095146
Data columns (total 49 columns):
 #   Column                                 Dtype         
---  ------                                 -----         
 0   UF                                     object        
 1   Region                                 object        
 2   ANF                                    int64         
 3   Municipality                           object        
 4   Neighborhood                           object        
 5   eNodeB                                 object        
 6   Cell                                   object        
 7   Data                                   datetime64[us]
 8   TIM_ACC (%)                            float64       
 9   TIM_ACC_ERAB (%)                       float64       
 10  TIM_ACC_ERAB_DEN (Unid)                float64       
 11  TIM_ACC_ERAB_NUM (Unid)                float64       
 12  TIM_ACC_ERRC (%)                       float64       
 1

In [19]:
df

,UF,Region,ANF,Municipality,Neighborhood,eNodeB,Cell,Data,TIM_ACC (%),TIM_ACC_ERAB (%),...,TIM_PRB_UTIL_MEAN_UL_DEN (Unid),TIM_PRB_UTIL_MEAN_UL_NUM (Unid),TIM_THROU_USER_PDCP_DL (Kbps),TIM_THROU_USER_PDCP_DL_DEN (s),TIM_THROU_USER_PDCP_DL_NUM (Kb),TIM_THROU_USER_PDCP_UL (Kbps),TIM_THROU_USER_PDCP_UL_DEN (s),TIM_THROU_USER_PDCP_UL_NUM (Kb),TIM_USERS_RRC_CONN_MEAN_SUM (Unid),TIM_VOLUME_DADOS_DL_ALLOP (KB)
0,RJ,TRJ,21,ITAGUAI,BRISA MAR,4G-ITAG04,4C-ITAG04-26-A,2023-08-21 00:00:00,100.000000,100.000000,...,100.0,1896.5,43380.117679,221721.0,9.618283e+09,4136.225515,980726.0,4.056504e+09,22.007,2.056179e+06
1,RJ,TRJ,21,ITAGUAI,BRISA MAR,4G-ITAG04,4C-ITAG04-26-A,2023-08-21 01:00:00,99.957983,100.000000,...,100.0,907.7,48305.929456,218163.0,1.053857e+10,1078.817356,440660.0,4.753917e+08,17.648,2.032176e+06
2,RJ,TRJ,21,ITAGUAI,BRISA MAR,4G-ITAG04,4C-ITAG04-26-A,2023-08-21 02:00:00,99.972900,99.973552,...,100.0,868.7,37038.490747,89867.0,3.328538e+09,779.964670,395812.0,3.087194e+08,14.263,9.917462e+05
3,RJ,TRJ,21,ITAGUAI,BRISA MAR,4G-ITAG04,4C-ITAG04-26-A,2023-08-21 03:00:00,100.000000,100.000000,...,100.0,921.7,48441.509566,128323.0,6.216160e+09,989.452390,414525.0,4.101528e+08,16.734,1.480551e+06
4,RJ,TRJ,21,ITAGUAI,BRISA MAR,4G-ITAG04,4C-ITAG04-26-A,2023-08-21 04:00:00,100.000000,100.000000,...,100.0,835.1,41713.196990,84385.0,3.519968e+09,2157.751530,186566.0,4.025631e+08,15.231,9.395394e+05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2095142,RJ,TRJ,24,PETROPOLIS,ZONA RURAL,NLPETR86,NLPETR86S,2023-10-15 19:00:00,100.000000,100.000000,...,75.0,488.8,17458.093960,894.0,1.560754e+07,1141.639963,18437.0,2.104842e+07,1.353,2.181361e+04
2095143,RJ,TRJ,24,PETROPOLIS,ZONA RURAL,NLPETR86,NLPETR86S,2023-10-15 20:00:00,100.000000,100.000000,...,75.0,486.2,31527.569818,1153.0,3.635129e+07,1813.479585,14842.0,2.691566e+07,1.031,5.715520e+04
2095144,RJ,TRJ,24,PETROPOLIS,ZONA RURAL,NLPETR86,NLPETR86S,2023-10-15 21:00:00,99.529412,99.577167,...,75.0,504.0,13978.546053,2432.0,3.399582e+07,696.590046,47237.0,3.290482e+07,1.935,1.863269e+04
2095145,RJ,TRJ,24,PETROPOLIS,ZONA RURAL,NLPETR86,NLPETR86S,2023-10-15 22:00:00,100.000000,100.000000,...,75.0,474.3,18678.644068,177.0,3.306120e+06,1212.526543,6951.0,8.428272e+06,0.834,2.227201e+03


In [20]:
na_counts = df.isna().sum()
na_counts_df = na_counts.reset_index()
na_counts_df.columns = ['Coluna', 'Qtd_NaNs']
print(na_counts_df)

                                   Coluna  Qtd_NaNs
0                                      UF         0
1                                  Region         0
2                                     ANF         0
3                            Municipality         0
4                            Neighborhood         0
5                                  eNodeB         0
6                                    Cell         0
7                                    Data         0
8                             TIM_ACC (%)     90173
9                        TIM_ACC_ERAB (%)     89334
10                TIM_ACC_ERAB_DEN (Unid)     39247
11                TIM_ACC_ERAB_NUM (Unid)     39247
12                       TIM_ACC_ERRC (%)     89923
13                TIM_ACC_ERRC_DEN (Unid)     39247
14                TIM_ACC_ERRC_NUM (Unid)     39247
15   TIM_BUFFER_MEAN_USER_DL_VOLTE (Unid)   2095147
16                    TIM_CQI_MEAN (Unid)     80653
17                TIM_CQI_MEAN_DEN (Unid)     39247
18          

In [21]:
# Dropa data por enquanto
df = df.drop(columns=['UF', 'Region', 'ANF', 'Municipality','Neighborhood','eNodeB', 'Cell', 'Data', 'TIM_BUFFER_MEAN_USER_DL_VOLTE (Unid)', 'TIM_DROP_ERAB (%)'])
df = df.dropna(axis=0)

In [22]:
df

,TIM_ACC (%),TIM_ACC_ERAB (%),TIM_ACC_ERAB_DEN (Unid),TIM_ACC_ERAB_NUM (Unid),TIM_ACC_ERRC (%),TIM_ACC_ERRC_DEN (Unid),TIM_ACC_ERRC_NUM (Unid),TIM_CQI_MEAN (Unid),TIM_CQI_MEAN_DEN (Unid),TIM_CQI_MEAN_NUM (Unid),...,TIM_PRB_UTIL_MEAN_UL_DEN (Unid),TIM_PRB_UTIL_MEAN_UL_NUM (Unid),TIM_THROU_USER_PDCP_DL (Kbps),TIM_THROU_USER_PDCP_DL_DEN (s),TIM_THROU_USER_PDCP_DL_NUM (Kb),TIM_THROU_USER_PDCP_UL (Kbps),TIM_THROU_USER_PDCP_UL_DEN (s),TIM_THROU_USER_PDCP_UL_NUM (Kb),TIM_USERS_RRC_CONN_MEAN_SUM (Unid),TIM_VOLUME_DADOS_DL_ALLOP (KB)
0,100.000000,100.000000,4826.0,4826.0,100.000000,2782.0,2782.0,11.083330,4362487.0,48350881.0,...,100.0,1896.5,43380.117679,221721.0,9.618283e+09,4136.225515,980726.0,4.056504e+09,22.007,2.056179e+06
1,99.957983,100.000000,3864.0,3864.0,99.957983,2380.0,2379.0,11.028623,3371790.0,37186201.0,...,100.0,907.7,48305.929456,218163.0,1.053857e+10,1078.817356,440660.0,4.753917e+08,17.648,2.032176e+06
2,99.972900,99.973552,3781.0,3780.0,100.000000,2295.0,2295.0,10.776692,2917609.0,31442173.0,...,100.0,868.7,37038.490747,89867.0,3.328538e+09,779.964670,395812.0,3.087194e+08,14.263,9.917462e+05
3,100.000000,100.000000,4015.0,4015.0,100.000000,2457.0,2457.0,10.880847,3526255.0,38368640.0,...,100.0,921.7,48441.509566,128323.0,6.216160e+09,989.452390,414525.0,4.101528e+08,16.734,1.480551e+06
4,100.000000,100.000000,3729.0,3729.0,100.000000,2354.0,2354.0,10.995667,3252175.0,35759832.0,...,100.0,835.1,41713.196990,84385.0,3.519968e+09,2157.751530,186566.0,4.025631e+08,15.231,9.395394e+05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2093804,99.245852,99.369483,793.0,788.0,100.000000,395.0,395.0,10.533240,1240197.0,13063293.0,...,75.0,577.3,15669.754913,3460.0,5.421735e+07,782.728210,91405.0,7.154527e+07,3.681,4.700844e+04
2093805,99.663300,99.573864,704.0,701.0,100.000000,330.0,330.0,9.914059,1089887.0,10805204.0,...,75.0,609.4,17883.794738,7678.0,1.373118e+08,914.588755,144814.0,1.324453e+08,3.120,1.236441e+05
2094472,100.000000,100.000000,257.0,257.0,100.000000,138.0,138.0,10.180183,582341.0,5928338.0,...,75.0,532.1,17515.130736,6930.0,1.213799e+08,462.760666,144994.0,6.709752e+07,1.603,5.922121e+04
2094789,100.000000,99.753695,406.0,405.0,100.000000,206.0,206.0,11.302893,469556.0,5307341.0,...,75.0,495.7,16357.270237,803.0,1.313489e+07,457.398356,33342.0,1.525058e+07,1.653,9.077438e+03


In [23]:
scaler = StandardScaler()
df_scaled = scaler.fit_transform(df)

# x′= (x−μ) / σ​
# μ = média da coluna
# σ = desvio padrão da coluna
# padroniza as distâncias dentro das features numéricas

In [24]:
import os
import math

# Calcular tamanho de cada partição (10% do total)
tamanho_particao = math.ceil(len(df) * 0.1)

# Criar partições sequenciais
for i in range(0, len(df), tamanho_particao):
    particao = df.iloc[i:i + tamanho_particao].copy()
    particao["cluster_dbscan"] = labels 
    
    dbscan = DBSCAN(eps=3, min_samples=2)
    labels = dbscan.fit_predict(particao)
    
    particao["cluster_dbscan"] = labels
    
    nome_arquivo = os.path.join("data", f"particao_{num_particao}.csv")
    particao.to_csv(nome_arquivo, index=False)
    print(f"Partição {num_particao} salva com {len(particao)} linhas")

C:\Users\PUC\AppData\Local\Temp\ipykernel_11616\1376473193.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  particao["cluster_dbscan"] = labels


Partição 1 salva com 148659 linhas


C:\Users\PUC\AppData\Local\Temp\ipykernel_11616\1376473193.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  particao["cluster_dbscan"] = labels


Partição 2 salva com 148659 linhas


C:\Users\PUC\AppData\Local\Temp\ipykernel_11616\1376473193.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  particao["cluster_dbscan"] = labels


...

Partição 8 salva com 148659 linhas


C:\Users\PUC\AppData\Local\Temp\ipykernel_11616\1376473193.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  particao["cluster_dbscan"] = labels


Partição 9 salva com 148659 linhas


C:\Users\PUC\AppData\Local\Temp\ipykernel_11616\1376473193.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  particao["cluster_dbscan"] = labels


Partição 10 salva com 148653 linhas


In [ ]:
particao1 = pd.read_csv(r"data/particao_1.csv", engine="pyarrow")
particao2 = pd.read_csv(r"data/particao_2.csv", engine="pyarrow")
particao3 = pd.read_csv(r"data/particao_3.csv", engine="pyarrow")
particao4 = pd.read_csv(r"data/particao_4.csv", engine="pyarrow")
particao5 = pd.read_csv(r"data/particao_5.csv", engine="pyarrow")
particao6 = pd.read_csv(r"data/particao_6.csv", engine="pyarrow")
particao7 = pd.read_csv(r"data/particao_7.csv", engine="pyarrow")
particao8 = pd.read_csv(r"data/particao_8.csv", engine="pyarrow")
particao9 = pd.read_csv(r"data/particao_9.csv", engine="pyarrow")
particao10 = pd.read_csv(r"data/particao_10.csv", engine="pyarrow")

### Clustering 

In [26]:
# Lista de caminhos das partições
particoes = [f"data/particao_{i}.csv" for i in range(1, 11)]

for caminho in particoes:
    # Ler partição
    df_particao = pd.read_csv(caminho, engine="pyarrow")

    # Selecionar apenas colunas numéricas, excluindo a coluna de cluster
    num_cols = df_particao.select_dtypes(include='number').columns
    if 'cluster_dbscan' in num_cols:
        num_cols = num_cols.drop('cluster_dbscan')

    # Escalar os dados
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(df_particao[num_cols])

    # Aplicar PCA
    pca = PCA(n_components=2)
    pca_result = pca.fit_transform(X_scaled)

    # Adicionar as colunas ao DataFrame
    df_particao['pca1'] = pca_result[:, 0]
    df_particao['pca2'] = pca_result[:, 1]

    # Salvar de volta a partição com as novas colunas
    df_particao.to_csv(caminho, index=False)

    print(f"✅ PCA salvo em {caminho}")

✅ PCA salvo em data/particao_1.csv
✅ PCA salvo em data/particao_2.csv
✅ PCA salvo em data/particao_3.csv
✅ PCA salvo em data/particao_4.csv
✅ PCA salvo em data/particao_5.csv
✅ PCA salvo em data/particao_6.csv
✅ PCA salvo em data/particao_7.csv
✅ PCA salvo em data/particao_8.csv
✅ PCA salvo em data/particao_9.csv
✅ PCA salvo em data/particao_10.csv


In [ ]:
if len(set(df['cluster_dbscan'])) > 1 and -1 not in set(df['cluster_dbscan']):
    sil_dbscan = silhouette_score(X_scaled, df['cluster_dbscan'])
    dbi_dbscan = davies_bouldin_score(X_scaled, df['cluster_dbscan'])
    ch_dbscan = calinski_harabasz_score(X_scaled, df['cluster_dbscan'])
else:
    sil_dbscan = dbi_dbscan = ch_dbscan = None

In [ ]:
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.scatter(df['pca1'], df['pca2'], c=df['cluster_dbscan'], cmap='viridis', s=100)
plt.title('PCA vs Cluster DBSCAN')
plt.xlabel('PCA 1')
plt.ylabel('PCA 2')
plt.show()